# **1IADT - Tech Challenge - Fase 4**
**Luiz Carvalho**

**Grupo:** 48

**GitHub:**

**Youtube:**

---

**O PROBLEMA**

O Tech Challenge desta fase será a criação de uma aplicação que utilize análise de vídeo. O seu projeto deve incorporar as técnicas de reconhecimento facial, análise de expressões emocionais em vídeos e detecção de atividades.

**A PROPOSTA DO DESAFIO**

Você deverá criar uma aplicação a partir do vídeo que se encontra disponível na plataforma do aluno, e que execute as seguintes tarefas:

1. **Reconhecimento facial**: Identifique e marque os rostos presentes no vídeo.
2. **Análise de expressões emocionais**: Analise as expressões emocionais dos rostos identificados.
3. **Detecção de atividades**: Detecte e categorize as atividades sendo realizadas no vídeo.
4. **Geração de resumo**: Crie um resumo automático das principais atividades e emoções detectadas no vídeo.


---

### **Instalando as bibliotecas**

In [ ]:
# %pip install opencv-python mediapipe python-docx tqdm deepface

### **Importando bibliotecas**

In [11]:
import cv2
import mediapipe as mp
from docx import Document
from tqdm import tqdm
from deepface import DeepFace

### **Inicia as variaveis de relatório**

In [12]:
# Variáveis globais
resumo_rostos_detectados = {}
resumo_emocoes_detectadas = {}
resumo_movimentos_detectados = {}
contagem_frames = 0

### **Inicia as variaveis de detecção**

In [13]:
# Carregar o classificador Haar Cascade para detecção de faces
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [14]:
# Carregar o modelo de detecção de pontos faciais e o modelo de detecção de poses do Mediapipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

### **1. Reconhecimento facial**

In [15]:
def detecta_rosto(frame):
    # Converter a imagem para tons de cinza
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detectar faces na imagem
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Para cada região detectada, tentar gerar um embedding e verificar se é um rosto real 
    for (x, y, w, h) in faces:
        # Recortar a região do rosto
        face_region = frame[y:y+h, x:x+w]

         # Validar se a região está dentro dos limites do frame
        if face_region.size == 0 or x < 0 or y < 0:
            resumo_rostos_detectados["invalidos"] += 1
            
            continue

        # Pré-processar o rosto para melhorar a qualidade
        face_region = cv2.resize(face_region, (160, 160))  # Redimensionar para a entrada esperada pelo modelo
        face_region = cv2.cvtColor(face_region, cv2.COLOR_BGR2RGB)  # Converter para RGB se necessário
        
        try:
            # Gerar o embedding usando DeepFace e o modelo Facenet
            embedding = DeepFace.represent(face_region, model_name="Facenet")
            
             # Adicionar validação com o tamanho do embedding para reduzir falsos positivos
            if embedding and len(embedding) > 0:
                # Desenhar um retângulo ao redor do rosto e marcar como válido
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, 'Valido', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                
                if resumo_rostos_detectados.get("validos") is None:
                    resumo_rostos_detectados["validos"] = 1
                else:
                    resumo_rostos_detectados["validos"] += 1
            else:
                if resumo_rostos_detectados.get("invalidos") is None:
                    resumo_rostos_detectados["invalidos"] = 1
                else:
                    resumo_rostos_detectados["invalidos"] += 1           
        except Exception as e:
            # Adicionar a contagem de rostos inválidos ao resumo
            if resumo_rostos_detectados.get("invalidos") is None:
                resumo_rostos_detectados["invalidos"] = 1
            else:
                resumo_rostos_detectados["invalidos"] += 1

### **2. Análise de expressões emocionais**

In [16]:
def detecta_emocoes(frame):
    # Processa o frame para detectar emoções
    result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
    
    # Loop para cada rosto detectado no frame
    for face in result:
        # Valida se a confiança da detecção é maior que 90%
        if face['face_confidence'] > 0.9:
            x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']
            dominant_emotion = face['dominant_emotion']
            
            # Registra a emoçção detectada para o resumo
            if dominant_emotion in resumo_emocoes_detectadas:
                resumo_emocoes_detectadas[dominant_emotion] += 1
            else:
                resumo_emocoes_detectadas[dominant_emotion] = 1
            
            # Desenha um retângulo ao redor do rosto e exibe a emoção detectada
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, dominant_emotion, (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,0,0), 2)

### **3. Detecção de atividades**

In [ ]:
def detectar_poses(frame):
    # Converte o quadro para RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detecta poses
    results = pose.process(frame_rgb)
    
    # Desenha as poses
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Obter os pontos de referência da pose
        landmarks = results.pose_landmarks.landmark
        
        # Obter coordenadas do ombro e pulso
        ombro_esquerdo = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        ombro_direito = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        pulso_esquerdo = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        pulso_direito = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        
        # Obter coordenadas do nariz
        nariz = landmarks[mp_pose.PoseLandmark.NOSE.value]
        
        # Valida se o braço esquerdo ou direito está levantado
        if pulso_esquerdo.y < ombro_esquerdo.y:
            if resumo_movimentos_detectados.get("braço_esquerdo_levantado") is None:
                resumo_movimentos_detectados["braço_esquerdo_levantado"] = 1
            else:
                resumo_movimentos_detectados["braço_esquerdo_levantado"] += 1              
        elif pulso_direito.y < ombro_direito.y:
            if resumo_movimentos_detectados.get("braço_direito_levantado") is None:
                resumo_movimentos_detectados["braço_direito_levantado"] = 1
            else:
                resumo_movimentos_detectados["braço_direito_levantado"] += 1
        # Valida se a mão esquerda ou direita está no rosto
        elif pulso_esquerdo.y < nariz.y:
            if resumo_movimentos_detectados.get("mão_esquerda_no_rosto") is None:
                resumo_movimentos_detectados["mão_esquerda_no_rosto"] = 1
            else:
                resumo_movimentos_detectados["mão_esquerda_no_rosto"] += 1
        elif pulso_direito.y < nariz.y:
            if resumo_movimentos_detectados.get("mão_direita_no_rosto") is None:
                resumo_movimentos_detectados["mão_direita_no_rosto"] = 1
            else:
                resumo_movimentos_detectados["mão_direita_no_rosto"] += 1
        # Caso nenhum movimento acima seja detectado, considerar como movimento anômalo
        else:
            if resumo_movimentos_detectados.get("movimento_anomalo") is None:
                resumo_movimentos_detectados["movimento_anomalo"] = 1
            else:
                resumo_movimentos_detectados["movimento_anomalo"] += 1

### **4. Geração de resumo**

In [ ]:
def gerar_resumo():
    # Cria um documento
    doc = Document()
    
    # Adiciona um título
    doc.add_heading('Resumo da Análise de Vídeo', level=1)
    
    # Adiciona os dados do resumo
    doc.add_heading('Contagem de frames', level=2)
    doc.add_paragraph(f"Total de frames: {contagem_frames}")
    
    doc.add_heading('Rostos detectados', level=2)
    for key, value in resumo_rostos_detectados.items():
        doc.add_paragraph(f"{key}: {value}")
        
    doc.add_heading('Emoções detectadas', level=2)
    for key, value in resumo_emocoes_detectadas.items():
        doc.add_paragraph(f"{key}: {value}")
        
    doc.add_heading('Movimentos detectados', level=2)
    for key, value in resumo_movimentos_detectados.items():
        doc.add_paragraph(f"{key}: {value}")
        
    # Salva o documento
    doc.save("resumo_analise_video.docx")

### **5. Análise do video**

In [ ]:
def processa_video(caminho_video, caminho_saida):
    global contagem_frames
    
    # Carrega o vídeo
    video = cv2.VideoCapture(caminho_video)
    
    if not video.isOpened():
        print("Erro ao abrir o vídeo")
        return
    else:
        print("Vídeo carregado com sucesso")
    
    # Obter informações do vídeo
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Processar um frame a cada x frames
    intervalo_entre_frames = 5
    
    # Configuração para salvar o vídeo processado
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(caminho_saida, fourcc, fps, (width, height))
    
    # Processar o vídeo
    for _ in tqdm(range(total_frames), desc="Processando vídeo"):
        ret, frame = video.read()
        
        if not ret:
            break
        
        # Processar o frame a cada intervalo definido
        if contagem_frames % intervalo_entre_frames == 0:
            # Detecta rostos
            detecta_rosto(frame)
            
            # Detecta emoções
            detecta_emocoes(frame)
            
            # Detecta poses
            detectar_poses(frame)
        
        # Escreve o quadro processado no arquivo de saída
        out.write(frame)
        
        # Adiciona 1 ao contador de frames
        contagem_frames += 1
    
    # Libera os recursos
    video.release()
    out.release()
    cv2.destroyAllWindows()
    
    print("Vídeo processado com sucesso")
    
    # Gera o documento de resumo
    gerar_resumo()

In [20]:
caminho_video = '../Video/video_tech_challenge.mp4'
caminho_saida = '../Video/video_tech_challenge_output.mp4'
processa_video(caminho_video, caminho_saida)

Vídeo carregado com sucesso


Processando vídeo:   0%|          | 0/3326 [00:00<?, ?it/s]

Processando vídeo: 100%|██████████| 3326/3326 [04:59<00:00, 11.10it/s]

Vídeo processado com sucesso
